<center><b> Anti-Fraud Ops Request | 30NOV23 </center>

Hi @Gaurav and Team,
 
 
Good afternoon.
 
Can you please help us extract the PIDs related to the JIRA tickets and TSAs in the attached file (Sheet2 and Sheet3)?

We need the data for an upcoming IT development related to fraud account tagging. Thank you.

Ler me know if you have questions.


Regards,
Mela


In [1]:
# Import Libraries

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

In [2]:
# Read the data from sheet from sheet2 and check the shape to make sure that the number of rows imported are equal to number of rows in the sheet.
dfinputs2 = pd.read_excel(r"D:\MyStuff\Mela Sombillo csombillo1@tonikbank.com\Anti-Fraud Ops Request  30NOV23\Fraud PIDs v1.xlsx", sheet_name='Sheet2')
dfinputs2.shape

(322, 3)

In [3]:
# Check the first 5 rows of the data imported
dfinputs2.head()

,Fraud Type,TSA,PID
0,Fraud Card Transaction,60810034620009,NaN
1,Fraud Card Transaction,60810055110009,NaN
2,Fraud Card Transaction,60810150300002,NaN
3,Fraud Card Transaction,60810286150008,NaN
4,Fraud Card Transaction,60810397930004,NaN


In [4]:
# Check the datatype and other details of the imported dataframe. Match it with the table description
dfinputs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Fraud Type  322 non-null    object 
 1   TSA         322 non-null    int64  
 2   PID         0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 7.7+ KB


In [5]:
# Since the table have datatype as string so changing the TSA column which is integer to string
dfinputs2['TSA'] = dfinputs2['TSA'].astype('str')

In [6]:
# Check the table information again to make sure the change intended is done
dfinputs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Fraud Type  322 non-null    object 
 1   TSA         322 non-null    object 
 2   PID         0 non-null      float64
dtypes: float64(1), object(2)
memory usage: 7.7+ KB


In [7]:
# Change the string value to Tuple
a = tuple(dfinputs2['TSA'])


In [8]:
# Run the query and check the first 5 rows of the output dataframe
sq = f"""select accountNumber, customerId from `risk_credit_mis.banking_report` where accountNumber in {a};"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.head

Job ID 9c358778-0037-40c5-b450-5eb978373dc7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


<bound method NDFrame.head of       accountNumber customerId
0    60811455090001    1145509
1    60820189960006    2018996
2    60815558320008    1555832
3    60816057620002    1605762
4    60816154040008    1615404
..              ...        ...
317  60812492520007    1249252
318  60821749690003    2174969
319  60822922060005    2292206
320  60819287440001    1928744
321  60810993000001    1099300

[322 rows x 2 columns]>

In [9]:
df2 = dfinputs2.merge(df, left_on='TSA', right_on='accountNumber', how='left')
df2.head()

,Fraud Type,TSA,PID,accountNumber,customerId
0,Fraud Card Transaction,60810034620009,NaN,60810034620009,1003462
1,Fraud Card Transaction,60810055110009,NaN,60810055110009,1005511
2,Fraud Card Transaction,60810150300002,NaN,60810150300002,1015030
3,Fraud Card Transaction,60810286150008,NaN,60810286150008,1028615
4,Fraud Card Transaction,60810397930004,NaN,60810397930004,1039793


In [10]:
df2.isnull().sum()

Fraud Type         0
TSA                0
PID              322
accountNumber      0
customerId         0
dtype: int64

In [11]:
b = df2[df2.accountNumber.isnull()]
b

,Fraud Type,TSA,PID,accountNumber,customerId


In [12]:
df2.head()

,Fraud Type,TSA,PID,accountNumber,customerId
0,Fraud Card Transaction,60810034620009,NaN,60810034620009,1003462
1,Fraud Card Transaction,60810055110009,NaN,60810055110009,1005511
2,Fraud Card Transaction,60810150300002,NaN,60810150300002,1015030
3,Fraud Card Transaction,60810286150008,NaN,60810286150008,1028615
4,Fraud Card Transaction,60810397930004,NaN,60810397930004,1039793


In [13]:
df2.drop(columns=['PID','accountNumber'], inplace = True)
df2.head()

,Fraud Type,TSA,customerId
0,Fraud Card Transaction,60810034620009,1003462
1,Fraud Card Transaction,60810055110009,1005511
2,Fraud Card Transaction,60810150300002,1015030
3,Fraud Card Transaction,60810286150008,1028615
4,Fraud Card Transaction,60810397930004,1039793


In [14]:
df2.rename(columns={'customerId':'PID'}, inplace = True)
df2.columns

Index(['Fraud Type', 'TSA', 'PID'], dtype='object')

In [15]:
df2.to_excel('Fraud_PID_v1_Sheet2data_20231205.xlsx', index = False, sheet_name='Data')